Notes:
- there are 60 episodes, so we need a maximum of 6 features

### 1. Importing the data

In [6]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [7]:
AVATAR_URL = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2020/2020-08-11/avatar.csv'
SCENE_URL = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2020/2020-08-11/scene_description.csv'

res_avatar = duckdb.sql(f"select * from '{AVATAR_URL}'")
res_scene = duckdb.sql(f"select * from '{SCENE_URL}'")

In [8]:
duckdb.sql(f"describe select * from '{AVATAR_URL}'").fetchall()

[('id', 'BIGINT', 'YES', None, None, None),
 ('book', 'VARCHAR', 'YES', None, None, None),
 ('book_num', 'BIGINT', 'YES', None, None, None),
 ('chapter', 'VARCHAR', 'YES', None, None, None),
 ('chapter_num', 'BIGINT', 'YES', None, None, None),
 ('character', 'VARCHAR', 'YES', None, None, None),
 ('full_text', 'VARCHAR', 'YES', None, None, None),
 ('character_words', 'VARCHAR', 'YES', None, None, None),
 ('writer', 'VARCHAR', 'YES', None, None, None),
 ('director', 'VARCHAR', 'YES', None, None, None),
 ('imdb_rating', 'VARCHAR', 'YES', None, None, None)]

In [9]:
duckdb.sql(f"describe select * from '{SCENE_URL}'").fetchall()

[('id', 'BIGINT', 'YES', None, None, None),
 ('scene_description', 'VARCHAR', 'YES', None, None, None)]

### 2. EDA

#### 2.1.1. Words counts per character by episode

In [10]:
res_word_counts = duckdb.sql(f"""
select 
    book_num, chapter_num, character
    , sum(len(character_words)) as word_counts
from '{AVATAR_URL}'
group by 
    book_num, chapter_num, character
""")

In [11]:
res_word_counts_normalized = duckdb.sql(f"""
SELECT 
    book_num,
    chapter_num,
    character,
    SUM(LENGTH(character_words)) AS char_word_count,
    SUM(LENGTH(character_words)) * 100.0 / SUM(SUM(LENGTH(character_words))) OVER (PARTITION BY book_num, chapter_num) AS normalized_word_count, 
    AVG(TRY_CAST(imdb_rating AS FLOAT)) as imdb_rating,
FROM '{AVATAR_URL}'
GROUP BY book_num, chapter_num, character
HAVING 
    char_word_count > 100
    and count(*) > 5
""")

In [12]:
res_word_counts_normalized.df()

,book_num,chapter_num,character,char_word_count,normalized_word_count,imdb_rating
0,1,15,Zuko,654.0,6.859660,7.9
1,1,15,Mother Superior,406.0,4.258443,7.9
2,1,15,Iroh,322.0,3.377386,7.9
3,1,15,Scene Description,126.0,1.321586,7.9
4,1,15,Aang,1319.0,13.834697,7.9
...,...,...,...,...,...,...
458,3,19,Bumi,667.0,8.840292,9.5
459,3,19,Zuko,1601.0,21.219351,9.5
460,3,19,Toph,334.0,4.426773,9.5
461,3,19,Sokka,520.0,6.891981,9.5


In [13]:
# res_word_counts_normalized.df().groupby(['book_num', 'chapter_num']).sum()

In [14]:
character_rating_from_normalized_words = duckdb.sql(f"""
WITH normalized_counts as (
    SELECT 
        book_num,
        chapter_num,
        character,
        SUM(LENGTH(character_words)) AS char_word_count,
        SUM(LENGTH(character_words)) * 100.0 / SUM(SUM(LENGTH(character_words))) OVER (PARTITION BY book_num, chapter_num) AS normalized_word_count, 
        AVG(TRY_CAST(imdb_rating AS FLOAT)) as imdb_rating,
    FROM '{AVATAR_URL}'
    GROUP BY book_num, chapter_num, character
    HAVING 
        char_word_count > 100
        and count(*) > 5
), character_rating as (
    SELECT
        character
        , SUM(normalized_word_count * imdb_rating) / SUM(normalized_word_count) as avg_rating
    from normalized_counts
    group by character
)

select * from character_rating
""")

In [15]:
character_rating_from_normalized_words.df().sort_values(by='avg_rating', ascending=True)[:50]

,character,avg_rating
3,Yue,5.963911
110,Zhao,7.031211
68,Canyon guide,7.100000
121,Zhang leader,7.100000
93,Gan Jin leader,7.100000
41,Gan Jin tribesman,7.100000
43,Tong,7.500000
77,Due,7.700000
71,Huu,7.700000
78,Tho,7.700000


In [16]:
# res_word_counts.df().sort_values(by='word_counts', ascending=True)[:300]

#### 2.1.2. Character Clustering based on word counts throughout the series

In [17]:
# row: character, word count episode

In [18]:
df_character_word_vector = pd.pivot_table(res_word_counts_normalized.df(), values=['normalized_word_count'], index=['character'], 
               columns=['book_num', 'chapter_num']).reset_index(drop=False).fillna(0.0) # drop=False

In [19]:
df_character_word_vector

character normalized_word_count                        \
book_num                                       1                         
chapter_num                                    1          2          3   
0                     Aang             22.573198  24.486191  23.818840   
1              Actor Sokka              0.000000   0.000000   0.000000   
2               Actor Zuko              0.000000   0.000000   0.000000   
3             Actress Aang              0.000000   0.000000   0.000000   
4            Actress Azula              0.000000   0.000000   0.000000   
..                     ...                   ...        ...        ...   
118                   Yung              0.000000   0.000000   0.000000   
119                    Zei              0.000000   0.000000   0.000000   
120           Zhang leader              0.000000   0.000000   0.000000   
121                   Zhao              0.000000   0.000000  18.477942   
122                   Zuko              8.794207  10.565189  12.344713   

                                                                              \
book_num                                                                       
chapter_num          4          5         6          7          8          9   
0            25.192774  34.644701  7.538701  33.473183  20.917059  17.141792   
1             0.000000   0.000000  0.000000   0.000000   0.000000   0.000000   
2             0.000000   0.000000  0.000000   0.000000   0.000000   0.000000   
3             0.000000   0.000000  0.000000   0.000000   0.000000   0.000000   
4             0.000000   0.000000  0.000000   0.000000   0.000000   0.000000   
..                 ...        ...       ...        ...        ...        ...   
118           0.000000   0.000000  0.000000   0.000000   0.000000   0.000000   
119           0.000000   0.000000  0.000000   0.000000   0.000000   0.000000   
120           0.000000   0.000000  0.000000   0.000000   0.000000   0.000000   
121           0.000000   0.000000  0.000000   0.000000   8.871313   0.000000   
122           7.050011   0.000000  0.000000   7.359111   7.855921  10.890628   

             ...                                                        \
book_num     ...          3                                              
chapter_num  ...         12         13         14        15         16   
0            ...  13.824212  29.143308   0.000000  0.000000  16.008598   
1            ...   0.000000   0.000000   0.000000  0.000000   0.000000   
2            ...   0.000000   0.000000   0.000000  0.000000   0.000000   
3            ...   0.000000   0.000000   0.000000  0.000000   0.000000   
4            ...   0.000000   0.000000   0.000000  0.000000   0.000000   
..           ...        ...        ...        ...       ...        ...   
118          ...   0.000000   0.000000   0.000000  0.000000   0.000000   
119          ...   0.000000   0.000000   0.000000  0.000000   0.000000   
120          ...   0.000000   0.000000   0.000000  0.000000   0.000000   
121          ...   0.000000   0.000000   0.000000  0.000000   0.000000   
122          ...  33.439299  43.123686  34.130294  9.409731  29.731870   

                                                                    
book_num                                                            
chapter_num         17         18         19         20         21  
0             7.630322  16.210215  26.136514   0.000000  15.506636  
1             7.799443   0.000000   0.000000   0.000000   0.000000  
2             5.153203   0.000000   0.000000   0.000000   0.000000  
3             8.018305   0.000000   0.000000   0.000000   0.000000  
4             4.317549   0.000000   0.000000   0.000000   0.000000  
..                 ...        ...        ...        ...        ...  
118           0.000000   0.000000   0.000000   0.000000   0.000000  
119           0.000000   0.000000   0.000000   0.000000   0.000000  
120           0.000000   0.000000   0.000000   0.000000   0.000

In [20]:
from sklearn.cluster import KMeans

# good: 12

kmeans = KMeans(n_clusters=20, init='k-means++', 
                n_init='auto', max_iter=300, verbose=0, random_state=None, copy_x=True, algorithm='lloyd') #tol=0.0001,
# kmeans.fit(df_character_word_vector)

In [21]:
X = df_character_word_vector.drop(columns=['character']) # group character from majority class (?)
pred_y = kmeans.fit_predict(X)

# 3. Visualize
# plt.figure(figsize=(8, 6))
# plt.scatter(X[:, 0], X[:, 1], c=pred_y, s=50, cmap='viridis')
# Plot centroids
# plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
#             s=200, c='red', marker='X', label='Centroids')
# plt.title("KMeans Visualization")
# plt.legend()
# plt.show()

/var/folders/rq/6nv9m_7j7wnb5x_4cztrwm0h0000gp/T/ipykernel_56718/2966549090.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = df_character_word_vector.drop(columns=['character']) # group character from majority class (?)


In [22]:
pred_y

array([ 2,  3,  3,  3,  3,  3,  3,  7,  3,  3, 11,  3,  3,  3,  3,  3,  3,
       10,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3, 17, 12,  3,  3,  3,  0,  3,  8,  3,  3, 16,  3,  3,  3,
        3,  6,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3, 15,  3,  3,
        3,  3,  3,  3,  3, 13, 19,  3,  3,  9,  3,  3,  3,  3,  3,  3,  3,
        5,  3,  9,  3,  1,  3,  3, 15,  3,  3,  3,  3,  3, 14,  1,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3, 18,  4], dtype=int32)

In [23]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=5, min_samples=2, algorithm='auto')
pred_y = dbscan.fit_predict(X)

In [24]:
pred_y

array([-1,  0,  0,  0,  0,  0, -1, -1, -1,  0, -1,  1,  2,  0, -1, -1, -1,
       -1,  0,  3,  4,  5,  0,  0,  0, -1, -1,  2,  2,  6, -1, -1,  6, -1,
        0, -1, -1, -1, -1,  0,  0,  7, -1, -1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1,  0,  0, -1, -1,  7,  0, -1,  6, -1, -1,  0,  0, -1,  1,  0,
        8,  5, -1, -1, -1, -1, -1,  8, -1, -1,  0, -1,  6,  5,  0,  0, -1,
       -1, -1, -1,  0, -1, -1,  0, -1,  0,  0,  4,  0, -1, -1, -1, -1,  0,
        0,  0, -1, -1, -1, -1,  0,  3,  0,  0, -1,  0, -1,  0, -1, -1,  0,
       -1,  2, -1, -1])

In [25]:
df_character_word_vector['character'].iloc[np.where(pred_y == 6)]

29    Gansu
32      Gow
60      Lee
80     Sela
Name: character, dtype: object

#### 2.1.3. 

In [26]:
main_characters = ['Aang', 'Katara', 'Sokka', 'Zuko', 'Toph']
secondary_characters = ['Iroh', 'Azula', 'Suki', 'Jet', 'Long Feng'] # between 50 and 100 words spoken
# other_characters = # 

### 3. Prediction - Episode Rating

### 3.1. Dummy - Mean Prediction by book

In [27]:
res_book_rating = duckdb.sql(f"""
select
    book_num, AVG(TRY_CAST(imdb_rating AS FLOAT)) as avg_rating
from '{AVATAR_URL}'
group by book_num
""")

In [28]:
res_book_rating.df()

,book_num,avg_rating
0,3,8.862371
1,1,8.287778
2,2,8.730077


In [29]:
res_book_rating = duckdb.sql(f"""
WITH chapter_avg AS (
    SELECT
        book_num,
        chapter_num,
        AVG(TRY_CAST(imdb_rating AS FLOAT)) AS chapter_avg_rating
    FROM '{AVATAR_URL}'
    WHERE
        imdb_rating not null
    GROUP BY book_num, chapter_num
),
book_avg AS (
    SELECT
        *,
        AVG(chapter_avg_rating) OVER (PARTITION BY book_num) AS book_avg_rating
    FROM chapter_avg
),
prediction_rating AS (
    SELECT
        (chapter_avg_rating - book_avg_rating) * 
        (chapter_avg_rating - book_avg_rating) AS mse
    FROM book_avg
)

SELECT
    SUM(mse) / COUNT(*) AS mse
FROM prediction_rating
""")

# select * from chapter_avg
# order by book_num, chapter_num


In [30]:
print(res_book_rating.df())

        mse
0  0.272674


In [31]:
print(res_avatar.df()['imdb_rating'].min())
print(res_avatar.df()['imdb_rating'].max())

7.1
NA


In [32]:
#### 3.1.2. Dummy - Mean Prediction by 

### 3.2. Based on writer/director (naive)

In [33]:
df_writer_director = res_avatar.df()[['book_num', 'chapter_num', 'writer', 'director', 'imdb_rating']].drop_duplicates()

In [34]:
# data cleaning
df_writer_director['writer'] = df_writer_director['writer'].str.split(', ')
df_writer_director['director'] = df_writer_director['director'].str.split(', ')

# 
df_writer_director = df_writer_director.loc[df_writer_director['imdb_rating'] != 'NA']

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_writer_director[['writer', 'director']], 
    df_writer_director['imdb_rating'], 
    stratify=df_writer_director[['book_num']], 
    test_size=0.2,
)

# X_val, X_test, y_val, y_test = train_test_split(
#     X_test, y_test, 
#     test_size=0.2,
# )

In [36]:
# --- encoder

from sklearn.preprocessing import MultiLabelBinarizer

# Writer encoder
mlb_writer = MultiLabelBinarizer()
writer_train_ohe = mlb_writer.fit_transform(X_train['writer'])
writer_test_ohe  = mlb_writer.transform(X_test['writer'])

# Director encoder
mlb_director = MultiLabelBinarizer()
director_train_ohe = mlb_director.fit_transform(X_train['director'])
director_test_ohe  = mlb_director.transform(X_test['director'])

/Users/emulie/Projects/tidytuesday/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:1007: UserWarning: unknown class(es) ['Andrew Huebner', 'Elizabeth Welch Ehasz (story editor)', 'Gary Scheppke', 'Giancarlo Volpe', 'Joann Estoesta', "John O'Bryan (story editor)", 'Justin Ridge', 'Lauren MacMullan', 'Lisa Wahlander', 'Matthew Hubbard', '\u200eMichael Dante DiMartino'] will be ignored
  warnings.warn(


In [37]:
writer_train_df = pd.DataFrame(
    writer_train_ohe,
    columns=[f"writer_{c}" for c in mlb_writer.classes_],
    index=X_train.index
)

writer_test_df = pd.DataFrame(
    writer_test_ohe,
    columns=[f"writer_{c}" for c in mlb_writer.classes_],
    index=X_test.index
)

director_train_df = pd.DataFrame(
    director_train_ohe,
    columns=[f"director_{c}" for c in mlb_director.classes_],
    index=X_train.index
)

director_test_df = pd.DataFrame(
    director_test_ohe,
    columns=[f"director_{c}" for c in mlb_director.classes_],
    index=X_test.index
)


In [38]:
X_train_final = pd.concat(
    [writer_train_df, director_train_df],
    axis=1
)

X_test_final = pd.concat(
    [writer_test_df, director_test_df],
    axis=1
)

In [39]:
### --- MODEL: xgboost
from sklearn.metrics import mean_squared_error

import xgboost as xgb

model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train_final, y_train)
pred = model.predict(X_test_final)

In [40]:
rmse = np.sqrt(mean_squared_error(y_test, pred))

In [41]:
rmse

0.8373823068963092

In [42]:
# X_test_final['rating'] = y_test
# X_test_final['rating_pred'] = pred
# X_test_final

In [43]:
### --- MODEL: Lasso Lars
from sklearn.linear_model import LassoLars

lasso = LassoLars(alpha=.1)
lasso.fit(X_train_final, y_train)
pred = lasso.predict(X_test_final)

In [44]:
rmse = np.sqrt(mean_squared_error(y_test, pred))

In [45]:
rmse

0.618665819189434

In [46]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_final, y_train)
pred = lr.predict(X_test_final)

rmse = np.sqrt(mean_squared_error(y_test, pred))
print(rmse)

0.6667184544128724


### 3.2. Based on writer/director (Target, Frequency Encoding)

In [51]:
df_writer_director = res_avatar.df()[['book_num', 'chapter_num', 'writer', 'director', 'imdb_rating']].drop_duplicates()

# data cleaning
df_writer_director['writer'] = df_writer_director['writer'].str.split(', ')
# df_writer_director['director'] = df_writer_director['director'].str.split(', ')

# 
df_writer_director = df_writer_director.loc[df_writer_director['imdb_rating'] != 'NA']
df_writer_director['director'] = df_writer_director['director'].astype(str)
df_writer_director['imdb_rating'] = df_writer_director['imdb_rating'].astype(float)

In [59]:
X = df_writer_director[['book_num', 'chapter_num', 'writer', 'director']]
y = df_writer_director[['imdb_rating']]
stratify_col = df_writer_director['book_num']



In [60]:
from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import TargetEncoder
from category_encoders import TargetEncoder


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4)
for i, (train_index, test_index) in enumerate(skf.split(X, stratify_col)):
    # --- split dataset
    X_train, X_test = X.iloc[train_index].copy(), X.iloc[test_index].copy()
    y_train, y_test = y.iloc[train_index].copy(), y.iloc[test_index].copy()

    # --- encoding
    NUM_WRITERS = 2

    # encoder_director
    encoder_director = TargetEncoder()
    X_train['director_encoded'] = encoder_director.fit_transform(X_train[['director']], y_train)
    X_test['director_encoded'] = encoder_director.transform(X_test[['director']])

    # encoder_writer
    writer_counts = X_train['writer'].explode().value_counts() #.reset_index()
    writer_freq = writer_counts.to_dict()
    X_train['writer_freq'] = X_train['writer'].apply(
        lambda writers: [writer_freq[w] if w in writer_freq else 0 for w in writers]
    )
    X_train['writer_freq'] = X_train['writer_freq'].apply(lambda x: sorted(x, reverse=True))
    X_train['writer_freq'] = X_train['writer_freq'].apply(lambda x: x[:NUM_WRITERS] if len(x) >= NUM_WRITERS else x + [0] * (NUM_WRITERS-len(x)))

    X_test['writer_freq'] = X_test['writer'].apply(
        lambda writers: [writer_freq[w] if w in writer_freq else 0 for w in writers]
    )
    X_test['writer_freq'] = X_test['writer_freq'].apply(lambda x: sorted(x, reverse=True))
    X_test['writer_freq'] = X_test['writer_freq'].apply(lambda x: x[:NUM_WRITERS] if len(x) >= NUM_WRITERS else x + [0] * (NUM_WRITERS-len(x)))

    writer_cols = [f"writer_freq_{i}" for i in range(NUM_WRITERS) ]
    X_train[writer_cols] = pd.DataFrame(X_train['writer_freq'].to_list(), index=X_train.index)
    X_test[writer_cols] = pd.DataFrame(X_test['writer_freq'].to_list(), index=X_test.index)

    # --- feature engineered datasets
    X_train_final = X_train[['book_num']]
    X_test_final = X_test[['book_num']]
    
    # X_train_final = X_train[['director_encoded']]
    # X_test_final = X_test[['director_encoded']]
    
    # X_train_final = X_train[['director_encoded'] + writer_cols]
    # X_test_final = X_test[['director_encoded'] + writer_cols]
    
    # X_train_final = X_train[['director_encoded', 'book_num'] + writer_cols]
    # X_test_final = X_test[['director_encoded', 'book_num'] + writer_cols]

    # --- model
    model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=5, learning_rate=0.1, random_state=42)
    model.fit(X_train_final, y_train)
    pred = model.predict(X_test_final)

    # X_test['pred'] = pred
    # X_test['imdb_rating'] = y_test
    # print(X_test[['imdb_rating', 'pred']])

    rmse = rmse = np.sqrt(mean_squared_error(y_test, pred))
    print(f"Fold {i}: {rmse}")

Fold 0: 0.5585597101256908
Fold 1: 0.3837047610503748
Fold 2: 0.6996145353654102
Fold 3: 0.5054715411479128
Fold 4: 0.6551363214563752


#### Cleaner Code

In [61]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import TargetEncoder
import mlflow
import mlflow.sklearn
from mlflow import MlflowClient

# Custom transformer for writer frequency encoding
class WriterFrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, num_writers=2):
        self.num_writers = num_writers
        self.writer_freq = None
    
    def fit(self, X, y=None):
        # Calculate writer frequencies from training data
        writer_counts = X['writer'].explode().value_counts()
        self.writer_freq = writer_counts.to_dict()
        return self
    
    def transform(self, X):
        X = X.copy()
        
        # Apply frequency encoding
        X['writer_freq'] = X['writer'].apply(
            lambda writers: [self.writer_freq.get(w, 0) for w in writers]
        )
        X['writer_freq'] = X['writer_freq'].apply(lambda x: sorted(x, reverse=True))
        X['writer_freq'] = X['writer_freq'].apply(
            lambda x: x[:self.num_writers] if len(x) >= self.num_writers 
            else x + [0] * (self.num_writers - len(x))
        )
        
        # Create separate columns for each writer frequency
        writer_cols = [f"writer_freq_{i}" for i in range(self.num_writers)]
        X[writer_cols] = pd.DataFrame(X['writer_freq'].to_list(), index=X.index)
        
        return X

# Custom transformer for feature selection
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_set='book_only'):
        """
        feature_set options:
        - 'book_only': ['book_num']
        - 'director_only': ['director_encoded']
        - 'director_writer': ['director_encoded'] + writer_cols
        - 'all': ['director_encoded', 'book_num'] + writer_cols
        """
        self.feature_set = feature_set
        self.num_writers = 2
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        writer_cols = [f"writer_freq_{i}" for i in range(self.num_writers)]
        
        if self.feature_set == 'book_only':
            return X[['book_num']]
        elif self.feature_set == 'director_only':
            return X[['director_encoded']]
        elif self.feature_set == 'director_writer':
            return X[['director_encoded'] + writer_cols]
        elif self.feature_set == 'all':
            return X[['director_encoded', 'book_num'] + writer_cols]
        else:
            raise ValueError(f"Unknown feature_set: {self.feature_set}")


In [62]:
import os
from dotenv import load_dotenv

load_dotenv()

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

experiment_name = "avatar_imdb_prediction"

# experiment_id = mlflow.create_experiment(experiment_name)

try:
    experiment_id = client.create_experiment(
        experiment_name,
        artifact_location=Path.cwd().joinpath("mlruns").as_uri(),
        tags={"purpose": "example", "dataset": "iris"}
    )
except Exception as e:
    # If the experiment already exists, get its ID
    experiment = client.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id


In [63]:
NUM_WRITERS = 2
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4)
feature_set = 'book_only'

xgb_params = {
    'objective': "reg:squarederror",
    'n_estimators': 5,
    'learning_rate': 0.1,
    'random_state': 42
}

all_rmse = []
all_mae = []

for i, (train_index, test_index) in enumerate(skf.split(X, stratify_col)):
    with mlflow.start_run(experiment_id=experiment_id, run_name=f"fold_{i}"):
        # Log parameters
        mlflow.log_param("fold", i)
        mlflow.log_param("feature_set", feature_set)
        mlflow.log_param("num_writers", NUM_WRITERS)
        mlflow.log_param("n_splits", 5)
        mlflow.log_params(xgb_params)
        
        # Split dataset
        X_train, X_test = X.iloc[train_index].copy(), X.iloc[test_index].copy()
        y_train, y_test = y[train_index], y[test_index]
        
        # Director encoding
        encoder_director = TargetEncoder()
        X_train['director_encoded'] = encoder_director.fit_transform(X_train[['director']], y_train)
        X_test['director_encoded'] = encoder_director.transform(X_test[['director']])
        
        # Create pipeline
        pipeline = Pipeline([
            ('writer_encoder', WriterFrequencyEncoder(num_writers=NUM_WRITERS)),
            ('feature_selector', FeatureSelector(feature_set=feature_set)),
            ('model', xgb.XGBRegressor(**xgb_params))
        ])
        
        # Fit and predict
        pipeline.fit(X_train, y_train)
        pred = pipeline.predict(X_test)
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, pred))
        mae = np.mean(np.abs(y_test - pred))
        
        all_rmse.append(rmse)
        all_mae.append(mae)
        
        # Log metrics
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("train_size", len(y_train))
        mlflow.log_metric("test_size", len(y_test))
        
        # Log model
        mlflow.sklearn.log_model(pipeline, f"model_fold_{i}")
        
        # Add tags
        mlflow.set_tag("model_type", "xgboost")
        mlflow.set_tag("pipeline", "sklearn")
        
        print(f"Fold {i}: RMSE = {rmse:.4f}, MAE = {mae:.4f}")

# Summary
print("\n=== Cross-Validation Summary ===")
print(f"Mean RMSE: {np.mean(all_rmse):.4f} (+/- {np.std(all_rmse):.4f})")
print(f"Mean MAE: {np.mean(all_mae):.4f} (+/- {np.std(all_mae):.4f})")

🏃 View run fold_0 at: https://mlflow-cloud-service-400512913085.us-central1.run.app/#/experiments/14/runs/14a973b713914ef38386ea99fd076c22
🧪 View experiment at: https://mlflow-cloud-service-400512913085.us-central1.run.app/#/experiments/14


KeyError: "None of [Index([ 0,  2,  3,  4,  6,  7,  8,  9, 10, 11, 13, 14, 15, 17, 18, 19, 20, 21,\n       22, 23, 24, 25, 26, 28, 29, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43,\n       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 57],\n      dtype='int64')] are in the [columns]"

### 3.3. Based on character word counts

In [ ]:
# 1: main_characters, 2: secondary_characters; 3: other; 4: book_num